In [ ]:
# You don't need these two lines if you have pip installed topologicpy
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

## Import the needed classes

In [ ]:
# Import the needed classes
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Color import Color
from topologicpy.Plotly import Plotly
from topologicpy.EnergyModel import EnergyModel
from topologicpy.Helper import Helper
print("\nDONE")

## Check software versions

In [ ]:
import sys
print("This workflow was designed and verified to work with python 3.11, TopologicPy 0.8.54 and OpenStudio SDK 3.10.0.\n")
py_version = sys.version.split()[0]
print("Python version:", py_version)

print("TopologicPy Version:", Helper.Version(check=False))
print("OpenStudio SDK Version:", EnergyModel.Version(check=False))
print(" ")
print("TopologicPy Version:", Helper.Version(check=True))
print("OpenStudio SDK Version:", EnergyModel.Version(check=True))
print("\nDONE")

## Set the input variables for path names and load geometries

In [ ]:
# Set the needed file paths. Change to match your computer paths
weatherFilePath = r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\assets\EnergyModel\GBR_London.Gatwick.037760_IWEC.epw"
designDayFilePath = r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\assets\EnergyModel\GBR_London.Gatwick.037760_IWEC.ddy"
osBinaryPath = r"C:\openstudio-3.10.0\bin\openstudio.exe"
outputFolder = r"C:\Users\sarwj\OneDrive - Cardiff University\Desktop\EnergyModels"
buildingPath = r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\assets\EnergyModel\TopologicBuilding.brep"
shadingPath = r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\assets\EnergyModel\TopologicShadingSurfaces.brep"
osModelPath=r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\assets\EnergyModel\OSMTemplate-OfficeBuilding-3.10.0.osm"
print("\nDONE")

## Load and set the building geometry, apertures (External Windows), and shading surfaces

In [ ]:
# BUILDING
building = Topology.ByBREPPath(buildingPath)

# APERTURES
# Decompose the building
d = CellComplex.Decompose(building)
# Specify a glazing (window to wall) ratio:
import math
gr = 0.25
# Create apertures (windows) by scaling the external vertical walls
# Get the external vertical faces
walls = d['externalVerticalFaces']
apertures = []
for wall in walls:
    centroid = Topology.Centroid(wall)
    aperture = Topology.Scale(wall, centroid, math.sqrt(gr), math.sqrt(gr), math.sqrt(gr))
    apertures.append(aperture)
building = Topology.AddApertures(building, apertures, subTopologyType="face")

# SHADING SURFACES
shadingSurfaces = Topology.ByBREPPath(shadingPath)

# DISPLAY RESULT
Topology.Show(building, shadingSurfaces, apertures)
print("\nDONE")

## Create an Energy Model

In [ ]:
#import openstudio

#ddyModel = openstudio.openstudioenergyplus.loadAndTranslateIdf(openstudio.toPath(designDayFilePath))

model = EnergyModel.ByTopology(building=building,
                               shadingSurfaces = shadingSurfaces,
                               osModelPath=osModelPath,
                               weatherFilePath=weatherFilePath,
                               designDayFilePath=designDayFilePath,
                               glazingRatio=0.25)

print("THIS IS A SNIPPET OF THE OSM MODEL TO VERIFY THAT IT WORKED.")
print(str(model)[:900])
print("\nDONE")

## Export the model to OSM and GBXML

In [ ]:
# Uncomment these files and change output path if you wish to export the model
#status01 = EnergyModel.ExportToOSM(model, outputFolder+"/EnergModel.osm", overwrite=True)
#status02 = EnergyModel.ExportToGbXML(model, outputFolder+"/EnergModel.xml", overwrite=True)


## Run the Simulation

In [ ]:
# Run the simulation
model = EnergyModel.Run(model, weatherFilePath=weatherFilePath,
                        osBinaryPath=osBinaryPath, outputFolder=outputFolder)
print("\nDONE")

## Get the space names used in the energy model

In [ ]:
# Get the topologies from the energy model. These have the correct dictionaries with space names etc.
topologies = EnergyModel.Topologies(model)
cells = topologies['cells']

# Get the required row names for the SQL query
names = []
for cell in cells:
    d = Topology.Dictionary(cell)
    name = Dictionary.ValueAtKey(d, 'TOPOLOGIC_name')
    name = name.upper()+"_THERMAL_ZONE"
    names.append(name)
print("\nDONE")

## Set the report, table, and column names to retrieve from the SQL data

In [ ]:
# Set the report, table, and column names to retrieve from the SQL data

# Here we specify three possible reports. We get them by index for convenience.
reports = ["HVACSizingSummary", "HVACSizingSummary", "SystemSummary"]
forStrings = ["Entire Facility", "Entire Facility", "Entire Facility"]
tableNames = ["Zone Sensible Cooling", "Zone Sensible Heating", "Time Not Comfortable Based on Simple ASHRAE 55-2004"]
columnNames = ["User Design Load", "User Design Load", "Summer or Winter Clothes"]
unitsList = ["W", "W", "hr"]

# Choose report 0, 1, or 2
index = 0 

reportName = reports[index]
reportForString = forStrings[index]
tableName = tableNames[index]
columnName = columnNames[index]
units = unitsList[index]

# Get the values from the SQL data
values = EnergyModel.Query(model,
                          reportName = reportName,
                          reportForString = reportForString,
                          tableName = tableName,
                          columnName = columnName,
                          rowNames = names,
                          units = units)
print("\nDONE")

## Apply the values to the bottom face(s) of each space/cell.

In [ ]:
# This section is to apply the values only to the bottom face of each cell.
faces = []
selectors = []
for i, cell in enumerate(cells):
    d = Cell.Decompose(cell)
    bottomFaces = d['bottomHorizontalFaces']
    for bottomFace in bottomFaces:
        d = Dictionary.ByKeysValues(["value", "label"], [values[i], str(round(values[i], 2))])
        faces.append(bottomFace)
        s = Face.InternalVertex(bottomFace)
        s = Topology.SetDictionary(s, d)
        selectors.append(s)
faceCluster = Cluster.ByTopologies(faces)
faceCluster = Topology.TransferDictionariesBySelectors(faceCluster, selectors, tranFaces=True, tolerance=0.0001)
print("\nDONE")

## Create a Plotly figure and draw it.

In [ ]:
# Create a Plotly figure and draw it.
# Create a cluster to represent the building envelope (translucent). This is just for visualisation.
envelopeCluster = Cluster.ByTopologies(cells+apertures+Topology.Faces(shadingSurfaces))
plotlyData = Plotly.DataByTopology(envelopeCluster, faceOpacity=0.1)

# Add the color-coded faces
colorScale = "thermal"
plotlyData += Plotly.DataByTopology(faceCluster, faceOpacity=1, faceGroupKey="value",
                              faceLabelKey="label", faceGroups=values,
                              colorScale=colorScale)
# Create a Plotly Figure
figure = Plotly.FigureByData(plotlyData, width=800, height=600)

# Add a colorbar
figure = Plotly.AddColorBar(figure, values=values, width=30, outlineWidth=1,
                            nTicks=len(values), title=tableName, subTitle=columnName,
                            units=units, colorScale=colorScale, mantissa=2)
# Show plotly figure
Plotly.Show(figure)
print("\nDONE")